In [2]:
import os, nltk
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
os.getcwd()

'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [3]:
def get_word2vec_embeddings(dataset:pd.DataFrame, verbose:bool=False, dim:int=200) :
    sentences = []
    for sentence in list(dataset["text"]) :
        sentences.append(sentence.split())
    if verbose : 
        print("No. of Sentences:", len(sentences))
        for sentence in sentences[ : 5] : print("   ", sentence)
    
    model = Word2Vec(sentences=sentences, vector_size=dim, window=5, min_count=1, workers=4)
    model.train(sentences, epochs=100, total_examples=model.corpus_count, total_words=model.corpus_total_words, compute_loss=True)
    
    words = model.wv.index_to_key
    if verbose : 
        print("No. of Words:", len(words))
        for word in words[ : 5] : print("   ", word)
    
    vectors = []
    for word in words :
        vectors.append(model.wv[word])
    vectors = np.array(vectors)
    
    embeddings = pd.DataFrame(vectors, index=words, columns=range(1, dim + 1))
    
    return embeddings

In [6]:
# Para datasets com k_folding
# Args
k = 10
dataset_folder = "TwitterAirlines"
dataset_name = "TweetsProcessed2"
dim = 768
# dataset_folder = "../resources/datasets/IMDBMovieReviews"
# dataset_name = "IMDB_MRProcessed"

# Embeddings
save_path = f"../resources/embeddings/{dataset_folder}"
if not os.path.exists(save_path) : os.makedirs(save_path)

folds = [pd.read_csv(f"../resources/datasets/{dataset_folder}/folds/{dataset_name}_Fold{i_fold + 1}.csv", index_col=0) for i_fold in range(k)]

fold_embeddings = []
for i_fold in range(k) :
    trainset = pd.concat(folds[ : i_fold] + folds[i_fold + 1 : ])
    embeddings = get_word2vec_embeddings(trainset, dim=dim)
    embeddings.to_csv(f"{save_path}/{dataset_name}_Fold{i_fold + 1}_dim{dim}.csv")
    fold_embeddings.append(embeddings)
fold_embeddings[0]

,1,2,3,4,5,6,7,8,9,10,...,759,760,761,762,763,764,765,766,767,768
<start>,0.162348,-0.013083,0.270375,-0.006800,-0.154958,0.000836,0.949331,0.094301,0.917247,-0.112517,...,-0.292734,-0.336455,0.156907,-0.381857,-0.171791,0.022964,-0.048263,0.721060,0.170086,-0.532377
to,-0.146746,-0.978823,-0.662116,-0.617198,0.672955,-0.096742,-0.205448,0.164526,0.289334,-0.279339,...,-1.005971,0.285438,0.083150,-0.156128,-0.301172,-1.526028,-0.563874,1.465820,-0.126245,0.039330
the,0.159312,-0.571116,-0.028081,0.940197,0.720740,0.258031,-0.129797,-0.600792,0.250873,-0.146196,...,-1.222579,-0.898868,-0.305763,-0.615076,1.132764,-0.395233,-1.390983,0.312815,0.737994,-0.223924
i,-0.126318,-0.602547,-0.344949,-0.118339,0.633940,0.410045,0.185085,0.478479,0.247471,0.251278,...,-1.768768,-0.535172,-1.046221,-0.529159,-0.154308,-0.863616,0.048823,1.142463,-0.691120,0.377527
a,0.231701,-0.715782,0.279830,-0.184970,-0.169028,0.845474,-0.349712,0.016954,-0.240780,-0.201188,...,-1.462302,0.413082,-0.390848,0.776547,1.261863,-1.157455,-0.736341,1.155399,-0.008388,-0.336132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
moves,-0.049147,-0.050850,0.008084,-0.053894,0.137232,0.040727,0.034119,0.056034,-0.038340,0.061714,...,0.034720,0.017197,-0.050953,-0.001141,0.001909,-0.049678,0.013046,-0.039630,0.037146,-0.042778
clockwork,-0.021937,-0.150440,-0.002780,0.096823,-0.047655,0.074883,-0.138332,-0.069714,-0.192553,-0.064525,...,0.005676,0.005784,-0.040880,-0.026413,-0.079422,0.055023,0.063406,0.112519,-0.076362,-0.013227
russ,-0.084211,-0.182126,0.037461,0.082263,0.122477,0.125133,-0.010065,0.036825,-0.079764,0.069168,...,0.137508,0.031503,0.053473,-0.021883,-0.013817,0.182770,-0.117682,0.003585,0.081427,0.013033
1848,0.072270,0.064576,0.036486,0.117622,0.201344,-0.002614,0.026699,-0.019276,-0.505121,0.118222,...,-0.055660,-0.061003,0.043918,-0.036959,-0.130250,0.095331,-0.062239,-0.004626,0.089472,-0.154744


In [3]:
# Para dataset simples
# Args
resource_folder_name = "../resources"
dataset_folder_name = "StanfordSentimentTreebank"
dataset_name = "SST2Processed2-train"
dim = 768

# Exec
trainset = pd.read_csv(f"{resource_folder_name}/datasets/{dataset_folder_name}/split/{dataset_name}.csv")
embeddings = get_word2vec_embeddings(trainset, dim=dim)
embeddings.to_csv(f"{resource_folder_name}/embeddings/{dataset_folder_name}/{dataset_name}_dim{dim}.csv")
embeddings

,1,2,3,4,5,6,7,8,9,10,...,759,760,761,762,763,764,765,766,767,768
<start>,-0.034255,0.169965,-0.363289,0.506442,-0.427822,-0.150762,0.148484,0.252551,-0.354949,-0.372175,...,-0.298481,0.413810,0.289104,-0.229494,0.547424,-0.645688,-0.039345,0.678340,-0.932603,-0.236194
the,0.217823,-0.161214,0.394072,0.068987,0.046122,0.154046,-0.454703,0.359115,0.021212,-0.538390,...,-0.001280,0.295897,0.398796,-1.331156,0.053657,-0.017822,0.128449,0.474252,-0.645320,1.193220
a,0.167699,-0.140574,-0.182011,0.753963,0.042203,0.163304,0.138383,0.016099,-0.236057,-0.259253,...,-0.174040,0.231640,0.223893,-0.948751,0.399870,-0.272608,-0.146616,0.261264,0.009641,0.473784
and,-0.104824,0.308404,-0.035850,0.195959,0.277350,1.062822,0.031705,0.053349,0.142076,-0.109975,...,0.008958,-0.329107,0.387872,-0.461119,0.581441,0.248812,-0.531152,-0.077007,-0.493601,0.707036
of,-0.189153,-0.038014,0.178236,-0.094203,0.081555,0.331777,-0.471609,-0.064213,-0.952977,0.116187,...,-0.088022,0.850869,0.420592,-0.957811,0.035724,-1.089710,-0.550777,1.008658,0.052567,0.297134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
weirder,0.013368,0.089305,0.110290,0.035707,-0.029785,0.110641,-0.129766,0.096415,-0.014302,-0.029318,...,-0.063590,-0.049419,0.003443,0.006120,0.051539,-0.043003,0.129099,-0.067617,-0.005751,0.006141
nurtures,-0.047219,0.003882,-0.042473,0.050664,0.008885,0.082935,-0.066954,-0.038774,0.083825,0.112258,...,0.077044,0.007705,-0.012869,0.062868,0.021358,0.081706,-0.111334,0.050834,0.060996,-0.072740
duckling,-0.010130,0.024509,0.053648,0.065153,0.075865,0.007628,0.061000,-0.092785,-0.055394,-0.200623,...,-0.058598,0.096407,-0.009398,-0.195503,-0.211286,0.058663,0.139345,-0.016107,0.003114,0.144125
regrets,0.011201,0.055083,0.041088,0.044559,0.040724,-0.094580,0.030258,-0.044762,0.002380,-0.000654,...,0.014300,-0.098278,-0.083174,0.045312,0.024754,-0.066009,-0.005088,-0.020318,-0.017686,-0.028166
